In [1]:
from telnetlib import STATUS
import psycopg2
import pandas as pd

username = 'piacobelli'
password = 'PAiac14-'
host = 'greenplum01.corral.tacc.utexas.edu'
port = 5432
database = 'uthealth'

connection = psycopg2.connect(
    host = host, 
    database = database, 
    user = username, 
    port = port, 
    password = password, 
    keepalives = 1, keepalives_idle = 100
)
connection.autocommit = True

C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\piacobelli\.conda\envs\piacobelliEnv\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [12]:

countDiffQuery = f'''
    
    UPDATE qa_reporting.iqvia_counts 
    SET row_count_diff = spc_row_count - row_count, row_count_diff_percentage = 100. * abs(spc_row_count - row_count) / row_count, last_updated=CURRENT_DATE; 
'''

with connection.cursor() as cursor: 
    cursor.execute(countDiffQuery)




In [13]:
enrollSynthAggQuery = f'''
    DELETE FROM qa_reporting.iqvia_counts WHERE table_name = 'enroll_synth' AND year IS NULL; 

    INSERT INTO qa_reporting.iqvia_counts
    (table_name, row_count, pat_count, spc_row_count)
    SELECT * FROM (
        SELECT 
            'enroll_synth' AS table_name,
            COUNT(*) AS row_count, 
            COUNT(DISTINCT pat_id) AS pat_count, 
            19147789 AS spc_row_count
        FROM iqvia.enroll_synth
    ) as insert_result; 
'''

enrollSynthDiffQuery = f'''
    UPDATE qa_reporting.iqvia_counts
    SET row_count_diff = spc_row_count - row_count, row_count_diff_percentage = 100. * abs(spc_row_count - row_count) / row_count, last_updated=CURRENT_DATE
    WHERE year IS NULL; 
'''

with connection.cursor() as cursor: 
    cursor.execute(enrollSynthAggQuery)
    cursor.execute(enrollSynthDiffQuery)

In [6]:
dervClaimNoUpdateQuery = f'''
    WITH derv_claimno_counts AS (
        SELECT 
            year, 
            COUNT(DISTINCT derv_claimno) AS derv_clms
        FROM dev.pi_iqvia_derv_claimno_new_all_yr
        GROUP BY year
    ) 
    UPDATE qa_reporting.iqvia_counts a
    SET derv_claimno_count = b.derv_clms 
    FROM derv_claimno_counts b
    WHERE a.year = b.year AND a.table_name = 'claims' ; 
'''

with connection.cursor() as cursor: 
    cursor.execute(dervClaimNoUpdateQuery)